In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from automatic_climb_detection.resample_dataframe import resample_dataframe

In [ ]:
file = "../data/TourDeFrance2022/stage-15-parcours.csv"
df = pd.read_csv(file, index_col=0)
df1km = resample_dataframe(df, interpolation_column="distance", interpolation_step=1000)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(18, 6))
plt.suptitle(file)
axs[0].plot(df["longitude"], df["latitude"], "-o", label="High resolution data")
axs[0].plot(df1km["longitude"], df1km["latitude"], "-o", label="Low resolution data")
axs[0].set(xlabel="Longitude", ylabel="Latitude")
axs[0].legend()
axs[1].plot(df["distance"], df["altitude"], "-o", label="High resolution data")
axs[1].plot(df1km["distance"], df1km["altitude"], "-o", label="Low resolution data")
axs[1].set(xlabel="Distance [m]", ylabel="Altitude [m]")
axs[1].legend()

In [ ]:
df1km